In [1]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import wfdb

import matplotlib as plot
from scipy.signal import find_peaks
import shutil
import psutil
import pandas as pd  


In [2]:
path="download/physionet.org/files/mimic3wdb/1.0/39"
dirs= glob.glob(path+"/*")

In [3]:
files=glob.glob(dirs[0]+"/*")

In [4]:
input_final=np.empty((0, 2))
output_final=np.empty((0, 1))
cur_dir=0
cur_file=0
for d in range(20,len(dirs)):
    files=glob.glob(dirs[d]+"/*")
    for f in range(len(files)):
        total, used, free = shutil.disk_usage("/")
        if free>30000000000 and psutil.virtual_memory()[2]<40:
            url=files[f].replace(".dat","")
            url=url.replace(".hea","")
            signals, fields = wfdb.rdsamp(url)
            ppg_i=fields['sig_name'].index("PLETH")
            ecg_i=fields['sig_name'].index("II")
            bp_i=fields['sig_name'].index("ABP")
            ppg_temp= np.array(signals[:,ppg_i]) 
            ecg_temp=np.array(signals[:,ecg_i])
            bp_temp=np.array(signals[:,bp_i])
            ppg_na= np.argwhere(np.isnan(ppg_temp))
            ecg_na= np.argwhere(np.isnan(ecg_temp))
            bp_na= np.argwhere(np.isnan(bp_temp))
            all_na = np.unique(np.concatenate((ppg_na,ecg_na,bp_na),0))
            ppg_temp=np.reshape(np.delete( ppg_temp, all_na), (-1, 1)) 
            ecg_temp= np.reshape(np.delete( ecg_temp, all_na), (-1, 1)) 
            bp_temp= np.reshape(np.delete( bp_temp, all_na), (-1, 1)) 
            inputs=np.hstack((ppg_temp,ecg_temp))
            input_final=np.append(input_final,inputs,axis=0)
            output_final=np.append(output_final,bp_temp,axis=0)
            cur_dir=d
            cur_file=f            
            print(d)
            print(f)
            print('The CPU usage is: ', psutil.cpu_percent(4))
            print('RAM memory % used:', psutil.virtual_memory()[2])

20
0
The CPU usage is:  13.3
RAM memory % used: 6.2
20
1
The CPU usage is:  0.1
RAM memory % used: 5.7
20
2
The CPU usage is:  0.0
RAM memory % used: 5.5
20
3
The CPU usage is:  0.0
RAM memory % used: 7.3
21
0
The CPU usage is:  0.0
RAM memory % used: 6.2
21
1
The CPU usage is:  0.1
RAM memory % used: 6.4
22
0
The CPU usage is:  0.0
RAM memory % used: 7.0
22
1
The CPU usage is:  0.0
RAM memory % used: 9.3
22
2
The CPU usage is:  0.1
RAM memory % used: 7.0
22
3
The CPU usage is:  0.0
RAM memory % used: 9.4
22
4
The CPU usage is:  0.0
RAM memory % used: 7.6
22
5
The CPU usage is:  0.0
RAM memory % used: 7.7
22
6
The CPU usage is:  0.1
RAM memory % used: 8.0
22
7
The CPU usage is:  0.2
RAM memory % used: 8.0
22
8
The CPU usage is:  0.0
RAM memory % used: 8.0
22
9
The CPU usage is:  0.0
RAM memory % used: 8.1
22
10
The CPU usage is:  0.1
RAM memory % used: 8.1
22
11
The CPU usage is:  0.0
RAM memory % used: 9.0
23
0
The CPU usage is:  0.0
RAM memory % used: 11.8
23
1
The CPU usage is:  0.0

In [5]:
input_final=np.reshape(input_final, (input_final.shape[0], 1, input_final.shape[1]))

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers

In [7]:
def Model(input_dim, activation, num_class):
    model = Sequential()
    
    model.add(GRU(256, input_shape=(1, 2),  dropout = 0.25, recurrent_dropout = 0.25))

    #model.add(LSTM(256, input_shape=(1, 2),  dropout = 0.25, recurrent_dropout = 0.25))
    
    model.add(Dense(256))
    model.add(Activation(activation))

    model.add(Dropout(0.5))

    model.add(Dense(128))
    model.add(Activation(activation))
    model.add(Dropout(0.5))


    
    model.add(Dense(num_class))    
    model.add(Activation('linear'))
    model.compile(loss='Huber',
                      optimizer=optimizers.Adam(lr = 0.001),
                      metrics=['MeanAbsoluteError']
                )
    return model

In [8]:
input_dim = input_final.shape[1]
activation = 'sigmoid'
classes = 1
model = Model(input_dim=input_dim, activation=activation, num_class=classes)

2022-06-03 07:13:57.279839: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-03 07:13:57.279920: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-31-40-90): /proc/driver/nvidia/version does not exist
2022-06-03 07:13:57.281663: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/ubuntu/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model_0 = model.fit(input_final, # using the first 1million rows for speed.
                    output_final,
                    epochs=5,
                    batch_size=128,
                    verbose = 1,
                    validation_split=0.1,
                    validation_data=None,
                    shuffle=True,
                    class_weight=None,
                    sample_weight=None,
                    initial_epoch=0,
                    steps_per_epoch=None,
                    validation_steps=None,
                    validation_batch_size=None,
                    validation_freq=1,
                    max_queue_size=10,
                    workers=1,
                    use_multiprocessing=False
                   )

2022-06-03 07:14:08.710588: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1660180512 exceeds 10% of free system memory.
2022-06-03 07:14:10.428107: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 830090256 exceeds 10% of free system memory.
2022-06-03 07:14:11.278494: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1660180512 exceeds 10% of free system memory.
2022-06-03 07:14:12.275099: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1660180512 exceeds 10% of free system memory.


Epoch 1/5


2022-06-03 07:14:15.804899: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1660180512 exceeds 10% of free system memory.


1621271/1621271 [==============================] - 25952s 16ms/step - loss: 15.5436 - mean_absolute_error: 16.0349 - val_loss: 17.3560 - val_mean_absolute_error: 17.8485
Epoch 2/5
1621271/1621271 [==============================] - 28080s 17ms/step - loss: 15.1326 - mean_absolute_error: 15.6241 - val_loss: 17.9140 - val_mean_absolute_error: 18.4063
Epoch 3/5
1621271/1621271 [==============================] - 27717s 17ms/step - loss: 15.0572 - mean_absolute_error: 15.5481 - val_loss: 17.9084 - val_mean_absolute_error: 18.4007
Epoch 4/5
1621271/1621271 [==============================] - 27687s 17ms/step - loss: 15.0186 - mean_absolute_error: 15.5104 - val_loss: 18.0504 - val_mean_absolute_error: 18.5427
Epoch 5/5
 953043/1621271 [================>.............] - ETA: 3:05:04 - loss: 15.0017 - mean_absolute_error: 15.4929